In [62]:
import itertools

#list of links

urls = ["https://www.derstandard.at/search?query=justiz&n=inland&fd=2000-01-01&td=2009-12-31&s=date&p=", 
        "https://www.derstandard.at/search?query=justiz&n=inland&fd=2010-01-01&td=2016-12-31&s=date&p=", 
        "https://www.derstandard.at/search?query=justiz&n=inland&fd=2017-01-01&td=2022-12-31&s=date&p=", 
        "https://www.derstandard.at/search?query=vfgh&n=inland&fd=2000-01-01&td=2009-12-31&s=date&p=", 
        "https://www.derstandard.at/search?query=vfgh&n=inland&fd=2010-01-01&td=2022-12-31&s=date&p="]

# need to combine urls_p and pagenumber
urls_complete = []

for url in urls:
    urls_complete.append([url+ str(i) for i in range(1,18+1)])

# unnest
urls_complete = list(itertools.chain(*urls_complete))

In [63]:
len(urls_complete)

90

In [64]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [80]:
#set driver
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [81]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

#open standard homepage
driver.get("https://www.derstandard.at")

#accept cookies
WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR,"iframe[title='SP Consent Message']")))
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,"//button[@title='Einverstanden']"))).click()

#deny push notifications
driver.switch_to.default_content()
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//button[text()='Nein, danke']"))).click()

True

In [68]:
# works well, but i wonder whether this could be done faster with swifter or multiprocess
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

links = []
for url in urls_complete:
    driver.get(url)
    
    # link
    try:
        a = driver.find_elements(By.XPATH, "//div[@class='teaser-inner']//child::a") 
        # getting all links per page
        links.append([lnk.get_attribute("href") for lnk in a])
    except NoSuchElementException:
        a = "NA"

links = list(itertools.chain(*links))

len(links)

8153

In [69]:
import pyarrow.feather as feather
import pandas as pd

df_links = pd.DataFrame(links)

feather.write_feather(df_links, 'links.feather')

In [70]:
import pyarrow.feather as feather
import pandas as pd
import random
from selenium.webdriver.common.by import By

df = pd.read_feather('links.feather')
df.rename(columns={0: "links"})

links = df[0].values.tolist()

,links
0,https://www.derstandard.at/story/1259282847861...
1,https://www.derstandard.at/story/1259282500288...
2,https://www.derstandard.at/story/1259282555505...
3,https://www.derstandard.at/story/1259282550923...
4,https://www.derstandard.at/story/1259282453470...
...,...
8148,https://www.derstandard.at/story/1262209548290...
8149,https://www.derstandard.at/story/1262209312051...
8150,https://www.derstandard.at/story/1262209253669...
8151,https://www.derstandard.at/story/1262209033945...


In [72]:
live = r'jetzt/livebericht'
links_short = links
livebericht = []

for elem in links_short:
        if live in elem:
            livebericht.append(elem)
        if live in elem:
            links_short.remove(elem)
            
len(links_short)
print(livebericht)

8109

['https://www.derstandard.at/jetzt/livebericht/2000025211222/oesterreich-im-menschenrechts-check', 'https://www.derstandard.at/jetzt/livebericht/2000006940276/betrugsprozess-gegen-peter-westenthaler', 'https://www.derstandard.at/jetzt/livebericht/2000003276135/prozess-gegen-anti-akademikerball-demonstranten-josef-s-wird-fortgesetzt', 'https://www.derstandard.at/jetzt/livebericht/1378248923947/finale-im-telekom-prozess-karin-gastinger-sagt-aus', 'https://www.derstandard.at/jetzt/livebericht/1373512925778/telekom-prozess-lobbyist-peter-hochegger-vor-gericht', 'https://www.derstandard.at/jetzt/livebericht/1334797225602/nachlese-livebericht-grasser-sieht-sich-durch-angaben-seines-ex-kabinettschefs-nicht-belastet', 'https://www.derstandard.at/jetzt/livebericht/1302745546121/nachlese-live-bericht-spindeleggers-umbau-fertig', 'https://www.derstandard.at/jetzt/livebericht/1293371146324/menschenrechtspruefung-spindelegger-menschenrechts-training-fuer-polizei-ausreichend', 'https://www.derstanda

In [ ]:
# DO NOT RUN ONLY DO SAVE CODE
# now working with soup get html of all pages
import requests
import random
from bs4 import BeautifulSoup

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36"}

linkstest = random.sample(links, 50)
soups = []
for link in linkstest:
    page = requests.get(link, headers = headers)
    soups.append(BeautifulSoup(page.content, "html.parser"))

In [ ]:
# DO NOT RUN ONLY DO SAVE CODE
len(soups)
len(linkstest)

In [ ]:
# DO NOT RUN ONLY DO SAVE CODE
# trying this with soup
from datetime import datetime
import pandas as pd
import numpy as np

start_time = datetime.now()

article = []
article2 = []
title = []
author = []
commentno = []
time = []

for soup in soups:
    article1 = soup.find('div', class_="article-body").findAll('p')
    #article.append(article1.text if article1 else "N/A")
    #article3 = article1.find('p')
    #article2.append(article3.text if article3 else "N/A")

    
    title1 = soup.find('h1', class_="article-title")
    title.append(title1.text if title1 else "N/A")
    
    author1 = soup.find('div', class_="article-origins")
    author.append(author1.text if author1 else "N/A")
    
    print(article1)
        
#df_articles=pd.DataFrame({'article': article,
 #                         'article2': article2,
  #                        'title': title,
   #                       'author': author})
    

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

In [87]:
# the rest with selenium
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime
import pandas as pd
import numpy as np

links1 = links_short[6001:-1]

df = pd.DataFrame(links1, columns = ['links'])

start_time = datetime.now()

def get_additions(row):        
    driver.get(row['links'])
    
    try:
        commentno = driver.find_element(By.CLASS_NAME, "article-postingcount").text
    except NoSuchElementException:
        commentno = "NA"
    try:
        time = driver.find_element(By.XPATH, "//time").text
    except NoSuchElementException:
        time = "NA"
    try:
        articles = driver.find_elements(By.CLASS_NAME, 'article-body')
        for value in articles:
            article = value.text
    except NoSuchElementException:
        article = "NA"
    try:
        titles = driver.find_elements(By.CLASS_NAME, 'article-title')
        for value in titles:
            title = value.text
    except NoSuchElementException:
        title = "NA"

    return pd.Series({
        'article': article,
        'title': title,
        'commentno': commentno,
        'time': time
    })

new_df2 = df.apply(get_additions, axis=1).join(df)
new_df2.head()

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

,article,title,commentno,time,links
0,Georg Bürstmayr ist Rechtsanwalt in Wien. Arbe...,<b>Einserfrage:</b> Ist das neue Asylgesetz di...,", \n14 Postings","3. Mai 2005, 18:42",https://www.derstandard.at/story/2035089/einse...
1,Wien - Der Unabhängige Bundesasylsenat (UBAS) ...,Bundesasylsenat bei Verfassungsgericht abgeblitzt,", \n2 Postings","3. Mai 2005, 13:27",https://www.derstandard.at/story/2022457/bunde...
2,Wien - Der Verfassungsjurist Dieter Kolonovits...,Auch Verfassungsrechtler gegen Haider-Vorschlag,", \n6 Postings","3. Mai 2005, 13:06",https://www.derstandard.at/story/2035151/auch-...
3,Tainach - Bundespräsident Heinz Fischer hat am...,"Fischer appelliert an ""guten Willen""",", \n12 Postings","3. Mai 2005, 10:05",https://www.derstandard.at/story/2034807/fisch...
4,Wien - Eine Reihe von Verfassungswidrigkeiten ...,"""Mehr Haft, mehr Härte, weniger Rechte""",", \n25 Postings","3. Mai 2005, 08:42",https://www.derstandard.at/story/1979505/mehr-...


Duration: 0:42:35.115536


In [74]:
feather.write_feather(new_df1, 'justiz1.feather')

In [88]:
feather.write_feather(new_df2, 'justiz4.feather')

In [21]:
from IPython.display import HTML

HTML(new_df.to_html(index=False))

### 